In [ ]:
import numpy as np
from jesse import helpers, research

_, trading_1m = research.get_candles(
    "Binance Spot",
    "BTC-USDT",
    "1m",
    helpers.date_to_timestamp("2020-01-01"),
    helpers.date_to_timestamp("2025-05-20"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

trading_1m.shape


In [2]:
np.save("data/btc_1m.npy", trading_1m)

In [ ]:
import numpy as np

candles = np.load("data/btc_1m.npy")
candles.shape

In [ ]:
# import os

# import pandas as pd
# from jesse.utils import numpy_candles_to_dataframe
# from pandarallel import pandarallel

# pandarallel.initialize(nb_workers=os.cpu_count() - 1, progress_bar=True)

# from custom_indicators.toolbox.entropy.apen_sampen import sample_entropy
# from custom_indicators.volitility_indicator.yang_zhang import yang_zhang_volatility

# df = numpy_candles_to_dataframe(candles)
# df["vol_150"] = yang_zhang_volatility(candles, period=150, sequential=True)
# df["vol_360"] = yang_zhang_volatility(candles, period=360, sequential=True)
# df["window"] = 150
# df["window_on_vol"] = df["window"] * df["vol_360"] / df["vol_150"]

# c_arr = df["close"].to_numpy()
# lag_arr = df["window_on_vol"].to_numpy()
# log_ret = [
#     np.log(c_arr[i] / c_arr[-round(lag) + i : i]) if np.isfinite(lag) else np.nan
#     for i, lag in zip(range(len(lag_arr)), lag_arr)
# ]
# df["log_ret"] = pd.Series(log_ret, index=df.index)
# df["samp_en"] = df["log_ret"].parallel_apply(
#     lambda x: sample_entropy(x) if isinstance(x, np.ndarray) else np.nan
# )

# df

In [1]:
import os

import numpy as np
import optuna
from mpire import WorkerPool
from numba import njit
from scipy import stats

from custom_indicators.toolbox.bar.build import build_bar_by_cumsum
from custom_indicators.toolbox.entropy.apen_sampen import sample_entropy_numba
from custom_indicators.volitility_indicator.yang_zhang import yang_zhang_volatility


@njit(cache=True)
def log_ret(candles: np.ndarray, window_on_vol: np.ndarray) -> list[np.ndarray]:
    log_ret_list = []
    for idx, w_on_vol in enumerate(window_on_vol):
        if np.isfinite(w_on_vol):
            w_on_vol = round(w_on_vol)
            if idx - w_on_vol >= 0:
                log_ret_list.append(
                    np.log(candles[idx, 2] / candles[idx - w_on_vol : idx, 2])
                )
    return log_ret_list


In [ ]:
def objective(trial: optuna.Trial) -> float:
    MIN_WINDOW = 20
    MAX_WINDOW = 60 * 24 * 7

    candles = np.load("data/btc_1m.npy")
    candles = candles[candles[:, 5] > 0]
    vol_t_window = trial.suggest_int("vol_t", 20, 720)
    vol_ref_window = trial.suggest_int("vol_ref", vol_t_window, 60 * 24 * 7)
    vol_t = yang_zhang_volatility(candles, period=vol_t_window, sequential=True)
    vol_ref = yang_zhang_volatility(candles, period=vol_ref_window, sequential=True)

    window = trial.suggest_int("window", 20, 720)
    window_on_vol = window * vol_ref / (vol_t + 1e-10)
    window_on_vol = np.clip(window_on_vol, MIN_WINDOW, MAX_WINDOW)

    log_ret_list = log_ret(candles, window_on_vol)
    with WorkerPool(n_jobs=os.cpu_count() - 1) as pool:
        entropy_array = pool.map(sample_entropy_numba, log_ret_list)

    len_gap = len(candles) - len(entropy_array)
    candles = candles[len_gap:]
    window_on_vol = np.round(window_on_vol[len_gap:])

    threshold_min = np.sum(entropy_array) / (len(candles) // 20)
    threshold_max = np.sum(entropy_array) / (len(candles) // 720)
    entropy_threshold = trial.suggest_float(
        "entropy_threshold", threshold_min, threshold_max
    )
    merged_bar = build_bar_by_cumsum(
        candles, entropy_array, entropy_threshold, reverse=False
    )
    if len(merged_bar) < len(candles) // 720:
        return 500
    close_arr = merged_bar[:, 2]
    ret = np.log(close_arr[1:] / close_arr[:-1])
    standard = (ret - ret.mean()) / ret.std()
    kurtosis = stats.kurtosis(standard, axis=None, fisher=False, nan_policy="omit")
    return kurtosis


TRIALS = 1000

study = optuna.create_study(
    direction="minimize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(n_startup_trials=100),
)
study.optimize(objective, n_trials=TRIALS, n_jobs=1, gc_after_trial=True)


In [ ]:
study.best_params

In [ ]:
"""
MIN_WINDOW = 20
MAX_WINDOW = 60 * 24
{'vol_t': 78, 'vol_ref': 1341, 'window': 134, 'is_div_window': False, 'entropy_threshold': 26.96225407898441}

MIN_WINDOW = 20å
MAX_WINDOW = 60 * 24 * 7
{'vol_t': 59, 'vol_ref': 7700, 'window': 42, 'entropy_threshold': 133.44931864959423}
"""

In [ ]:
from jesse import helpers, research

_, candles = research.get_candles(
    "Binance Spot",
    "BTC-USDT",
    "1m",
    helpers.date_to_timestamp("2020-01-01"),
    helpers.date_to_timestamp("2025-01-01"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)
candles = candles[candles[:, 5] > 0]
MIN_WINDOW = 20
MAX_WINDOW = 60 * 24

vol_t_window = 78
vol_ref_window = 1341
window = 134
entropy_threshold = 26.96225407898441

vol_t = yang_zhang_volatility(candles, period=vol_t_window, sequential=True)
vol_ref = yang_zhang_volatility(candles, period=vol_ref_window, sequential=True)

window_on_vol = window * vol_ref / (vol_t + 1e-10)
window_on_vol = np.clip(window_on_vol, MIN_WINDOW, MAX_WINDOW)

log_ret_list = log_ret(candles, window_on_vol)
with WorkerPool(n_jobs=os.cpu_count() - 1) as pool:
    entropy_array = pool.map(sample_entropy_numba, log_ret_list)

len_gap = len(candles) - len(entropy_array)
candles = candles[len_gap:]
window_on_vol = np.round(window_on_vol[len_gap:])

threshold_min = np.sum(entropy_array) / (len(candles) // 20)
threshold_max = np.sum(entropy_array) / (len(candles) // 360)
merged_bar = build_bar_by_cumsum(
    candles, entropy_array, entropy_threshold, reverse=False
)
close_arr = merged_bar[:, 2]
ret = np.log(close_arr[1:] / close_arr[:-1])
standard = (ret - ret.mean()) / ret.std()
kurtosis = stats.kurtosis(standard, axis=None, fisher=False, nan_policy="omit")
print(kurtosis)

In [ ]:
merged_bar[-1, 0]

In [ ]:
from jesse import helpers, research

from custom_indicators.toolbox.bar.entropy_bar_v2 import EntropyBarContainer

entropy_bar_container = EntropyBarContainer(134, 78, 1341, 26.96225407898441)

_, candles = research.get_candles(
    "Binance Spot",
    "BTC-USDT",
    "1m",
    helpers.date_to_timestamp("2020-01-01"),
    helpers.date_to_timestamp("2025-05-20"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)
candles = candles[candles[:, 5] > 0]

entropy_bar_container.update_with_candle(candles)
entropy_bar_container.get_entropy_bar()[-1, 0]

In [ ]:
merged_bar = entropy_bar_container.get_entropy_bar()
close_arr = merged_bar[:, 2]
ret = np.log(close_arr[1:] / close_arr[:-1])
standard = (ret - ret.mean()) / ret.std()
kurtosis = stats.kurtosis(standard, axis=None, fisher=False, nan_policy="omit")
kurtosis

In [ ]:
from custom_indicators.utils.plot import plot_kde

plot_kde(merged_bar[:, 2])

In [ ]:
from jesse.utils import numpy_candles_to_dataframe
from lightweight_charts import Chart

df_merged_bar = numpy_candles_to_dataframe(merged_bar)

chart = Chart()
chart.set(df_merged_bar[["date", "open", "high", "low", "close", "volume"]])
chart.show(block=True)
